<a href="https://colab.research.google.com/github/reckoning-machines/sec_text_nlp/blob/master/r_getter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext rpy2.ipython

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [23]:

%%R
devtools::install_github("mwaldstein/edgarWebR")
devtools::install_github("tidyverse/googlesheets4")
devtools::install_version("xml2", version = "1.2.2", repos = "http://cran.us.r-project.org")
#file creates a set of csv from ticker list which include metadata & text data.
library(edgarWebR) #this is an up to date library with an active maintainer.
library(xml2)
library(knitr)
library(dplyr)
library(purrr)
library(rvest)
library(tidyr)
library(readr)
#library(log4r) TODO logging file.
library(googlesheets4)
gs4_deauth()
#tickers list
str_sheet <- "1_xcDVKjR2jqE-w5LqxIWnrDYstpSrE5nFSSY0WXNOVE"
df_tickers <- read_sheet(str_sheet)


R[write to console]: Skipping install of 'edgarWebR' from a github remote, the SHA1 (e7fa70ea) has not changed since last install.
  Use `force = TRUE` to force installation

R[write to console]: Skipping install of 'googlesheets4' from a github remote, the SHA1 (edb84304) has not changed since last install.
  Use `force = TRUE` to force installation

R[write to console]: Downloading package from url: http://cran.us.r-project.org/src/contrib/Archive/xml2/xml2_1.2.2.tar.gz



R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: Reading from "test_ticker_list"

R[write to console]: Range "section_data"



In [24]:
%%R
company_filings("AXP", type = "10-", count = 20)

R[write to console]: Error in xml2::url_absolute(res[[ref]], xml2::xml_url(doc)) : 
  Base URL must be length 1
Calls: <Anonymous> ... company_filings -> map_xml -> ifelse -> <Anonymous>




Error in xml2::url_absolute(res[[ref]], xml2::xml_url(doc)) : 
  Base URL must be length 1
Calls: <Anonymous> ... company_filings -> map_xml -> ifelse -> <Anonymous>


In [15]:
%%R
#file creates a set of csv from ticker list which include metadata & text data.
df_tickers <- head(df_tickers,1)

get_filings_links <-function(str_ticker) {
  df_filings <- company_filings(str_ticker, type = "10-", count = 20)
  df_filings <- df_filings[df_filings$type == "10-K" | df_filings$type == "10-Q", ]
  df_filing_infos <- map_df(df_filings$href, filing_information)
  df_filings <- bind_cols(df_filings, df_filing_infos)
  return(head(as_tibble(df_filings),6))
}

get_section_text <- function(str_href, str_section, str_search) {
  df_filing_documents <- filing_documents(str_href)
  str_doc_href <- df_filing_documents[df_filing_documents$type == "10-K" | df_filing_documents$type == "10-Q",]$href
  doc <- parse_filing(str_doc_href)

  df_txt <- doc[grepl(str_section, doc$item.name, ignore.case = TRUE) & grepl(str_search, doc$item.name, ignore.case = TRUE), ] # only discussion for now
  #we could do some text preprocessing here.

  df_txt <- as_tibble(df_txt) %>%
    mutate(section = str_search)

  return(df_txt)
}

get_document_text <- function(str_ticker, force = FALSE) { #not using force yet
  start_time <- Sys.time()
  
  print(str_ticker)
  
  str_write_name <- paste0("~/data/",str_ticker,"_sec_text")

  df_filings <- get_filings_links(str_ticker)

  #print(df_filings)

  df_data <- (df_filings) %>% 
    rowwise() %>%
    mutate(nest_discussion = map(.x = href, str_section = 'item 2|item 7',str_search = 'discussion', .f = get_section_text)) %>%
    mutate(nest_qualitative = map(.x = href, str_section = 'item 3|item 7a', str_search = 'qualitative', .f = get_section_text)) %>%
    mutate(nest_controls = map(.x = href, str_section = 'item 4|item 9a',str_search = 'controls', .f = get_section_text)) %>%
    mutate(nest_risk = map(.x = href, str_section = 'item 1',str_search = 'risk factors', .f = get_section_text)) %>%
    ungroup() %>%
    select(period_date,filing_date,type,form_name,documents,nest_discussion,nest_qualitative,nest_controls,nest_risk) %>%
    group_by(period_date) %>%
    arrange(desc(period_date))
  
  #jenky - find a rowwise application
  a <- df_data %>% 
    select(period_date,filing_date,type,form_name,documents,nest_discussion) %>%
    unnest(nest_discussion)
  b <- df_data %>% 
    select(period_date,filing_date,type,form_name,documents,nest_qualitative) %>%
    unnest(nest_qualitative)
  c <- df_data %>% 
    select(period_date,filing_date,type,form_name,documents,nest_controls) %>%
    unnest(nest_controls)
  d <- df_data %>% 
    select(period_date,filing_date,type,form_name,documents,nest_risk) %>%
    unnest(nest_risk)
  
  ss <- gs4_create(str_write_name)
  
  df_data <- rbind(a,b,c,d) %>%
    googlesheets4::sheet_write(ss, sheet = "sec_data")
  
  end_time <- Sys.time()
  print(end_time - start_time)
  return(df_data)
}

#long run.
df_data <- map_df(df_tickers$Symbol, get_document_text)





[1] "GOOG"


R[write to console]: Error in xml2::url_absolute(res[[ref]], xml2::xml_url(doc)) : 
  Base URL must be length 1
Calls: <Anonymous> ... company_filings -> map_xml -> ifelse -> <Anonymous>




Error in xml2::url_absolute(res[[ref]], xml2::xml_url(doc)) : 
  Base URL must be length 1
Calls: <Anonymous> ... company_filings -> map_xml -> ifelse -> <Anonymous>
